In [7]:
import numpy as np
import pandas as pd
import altair as alt

In [8]:
# load in data
# opioid_deaths = pd.read_parquet("/Users/erikafox/Desktop/pds2021-opioids-pds6/10_modified_data/vital_pop.parquet", engine='fastparquet')
# opioid_buyers = pd.read_parquet("/Users/erikafox/Desktop/pds2021-opioids-pds6/10_modified_data/opi_pop.parquet", engine='fastparquet')

## load data for preet
opioid_deaths = pd.read_parquet("/Users/preetkhowaja/Documents/MIDS Fall 2021/Practicing DS/pds2021-opioids-pds6/10_modified_data/vital_pop.parquet", engine='fastparquet')
opioid_buyers = pd.read_parquet("/Users/preetkhowaja/Documents/MIDS Fall 2021/Practicing DS/pds2021-opioids-pds6/10_modified_data/opi_pop.parquet", engine='fastparquet')


In [9]:

opioid_buyers=opioid_buyers.rename(columns={"MME per cap":"MME_per_cap"})
opioid_deaths=opioid_deaths.rename(columns={"vital deaths per cap":"deaths_per_cap"})
opioid_buyers.sample(10)


,BUYER_STATE,BUYER_COUNTY,fips,year,MME,Population,MME_per_cap,treatment
index,,,,,,,,
1324,GA,BRYAN,13029,2009,1.167507e+07,29987.0,389.337554,0
6942,TX,FAYETTE,48149,2008,4.577070e+06,24347.0,187.993166,0
427,AL,MARSHALL,01095,2010,6.214595e+07,93019.0,668.099512,0
2452,GA,WALKER,13295,2007,1.848821e+07,67405.0,274.285398,0
6924,TX,FALLS,48145,2008,1.984921e+06,17698.0,112.155111,0
696,FL,COLLIER,12021,2009,2.029957e+08,318485.0,637.379143,1
7359,TX,JOHNSON,48251,2011,5.195160e+07,152094.0,341.575590,0
2147,GA,OGLETHORPE,13221,2010,3.034230e+06,14899.0,203.653258,0
6270,SC,ORANGEBURG,45075,2009,2.868003e+07,92755.0,309.202016,0


In [10]:
opioid_deaths.sample(10)

,STNAME,CTYNAME,fips,Year,Deaths,deaths_per_cap,treatment
index,,,,,,,
1858,Georgia,DeKalb County,13089,2011,44,0.000063,0
4923,California,Napa County,06055,2013,12,0.000086,0
1324,Oregon,Clackamas County,41005,2015,26,0.000065,0
477,Massachusetts,Berkshire County,25003,2010,10,0.000076,0
3034,Texas,Hidalgo County,48215,2004,11,0.000017,1
1328,Washington,Clallam County,53009,2005,10,0.000146,1
2250,Alabama,Etowah County,01055,2009,14,0.000134,0
5561,North Carolina,Pitt County,37147,2008,16,0.000099,0
2307,Kentucky,Fayette County,21067,2006,28,0.000100,0


In [11]:
## Nick's code for confidence bands 

def get_reg_fit(data, yvar, xvar, alpha=0.05,col="blue"):
    import statsmodels.formula.api as smf

    # Grid for predicted values
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})

    # Fit model, get predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg = alt.Chart(predictions).mark_line(color=col).encode(
        x=alt.X(xvar, axis=alt.Axis(title='Years from Policy Change')),
        y=alt.X(yvar, axis=alt.Axis(title='')))
    ci = (
        alt.Chart(predictions)
        .mark_errorband(color=col)
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=yvar),
            y2="ci_high",
        )
    )
    chart = ci + reg
    return predictions, chart

# Florida plots

In [12]:
# Creating florida buyers df


states = ["SC","AL","GA","MS", "FL"]

opioid_buyers_fl=opioid_buyers[opioid_buyers["BUYER_STATE"].isin(states)]

#create column, to indicate whether control or not (1 for control, 0 for florida)

opioid_buyers_fl["is_control"]=np.where(opioid_buyers_fl["BUYER_STATE"]== 'FL', 0, 1)

#create policy years 

opioid_buyers_fl["policy_years"]= opioid_buyers_fl["year"]-2010
fl_opi_before=opioid_buyers_fl[opioid_buyers_fl["policy_years"]<0]
fl_opi_after=opioid_buyers_fl[opioid_buyers_fl["policy_years"]>=0]

<ipython-input-12-6d09ea501f31>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_buyers_fl["is_control"]=np.where(opioid_buyers_fl["BUYER_STATE"]== 'FL', 0, 1)
<ipython-input-12-6d09ea501f31>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_buyers_fl["policy_years"]= opioid_buyers_fl["year"]-2010


In [13]:
#create vertical line for plots
data = pd.DataFrame({"a": [0]})
sep_line = (alt.Chart(data).mark_rule(color="black", strokeDash=[10, 10]).encode(x=alt.X("a:Q", title="")))

In [14]:
# florida buyers pre/post plot
before=alt.Chart(fl_opi_before,title="Pre-Post Policy: Florida, Opioids per cap").encode(x="policy_years", y="MME_per_cap", color= "is_control")
base=before.transform_regression("policy_years","MME_per_cap").mark_line() 

after=alt.Chart(fl_opi_after,title="Pre-Post Policy: Florida, Opioids per cap").encode(x="policy_years", y="MME_per_cap", color= "is_control")
base2=after.transform_regression("policy_years","MME_per_cap").mark_line() 

fit, before_line = get_reg_fit(
    fl_opi_before[fl_opi_before["is_control"]==1], yvar="MME_per_cap", xvar="policy_years", alpha=0.05, col="red"
)
fit, before_line2 = get_reg_fit(
    fl_opi_before[fl_opi_before["is_control"]==0], yvar="MME_per_cap", xvar="policy_years", alpha=0.05 
)
fit, after_line = get_reg_fit(
    fl_opi_after[fl_opi_after["is_control"]==1], yvar="MME_per_cap", xvar="policy_years", alpha=0.05, col="red"
)
fit, after_line2 = get_reg_fit(
    fl_opi_after[fl_opi_after["is_control"]==0], yvar="MME_per_cap", xvar="policy_years", alpha=0.05
)

fl_chart_opioid=base+base2+before_line+before_line2+after_line+after_line2+sep_line
fl_chart_opioid


alt.LayerChart(...)

In [15]:
# Creating florida deathd df

opioid_deaths_fl=opioid_deaths[opioid_deaths["STNAME"]=="Florida"]

opioid_deaths_fl["policy_years"]= opioid_deaths_fl["Year"]-2010


fl_death_before=opioid_deaths_fl[opioid_deaths_fl["policy_years"]<0]
fl_death_after=opioid_deaths_fl[opioid_deaths_fl["policy_years"]>=0]

<ipython-input-15-931f456a7cfb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_deaths_fl["policy_years"]= opioid_deaths_fl["Year"]-2010


In [16]:
# florida deaths pre/post plot
before=alt.Chart(fl_death_before,title="Pre-Post Policy: Florida, Overdose Deaths per cap").encode(x="policy_years", y="deaths_per_cap")
base=before.transform_regression("policy_years","deaths_per_cap").mark_line() 

fit, before_line = get_reg_fit(
    fl_death_before, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05
)
fit, after_line = get_reg_fit(
    fl_death_after, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05
)

fl_chart_death=base+before_line+after_line+sep_line
fl_chart_death

alt.LayerChart(...)

In [17]:
# Creating control dataframes
states = ["SC","AL","GA","MS"]
states2 = ["South Carolina","Alabama","Georgia","Mississippi"]

opioid_buyers_con=opioid_buyers[opioid_buyers["BUYER_STATE"].isin(states)]
opioid_buyers_con.sample(10)
opioid_buyers_con["policy_years"]= opioid_buyers_con["year"]-2010

con_opi_before=opioid_buyers_con[opioid_buyers_con["policy_years"]<0]
con_opi_after=opioid_buyers_con[opioid_buyers_con["policy_years"]>=0]


opioid_deaths_con=opioid_deaths[opioid_deaths["STNAME"].isin(states2)]
opioid_deaths_con["policy_years"]= opioid_deaths_con["Year"]-2010


con_death_before=opioid_deaths_con[opioid_deaths_con["policy_years"]<0]
con_death_after=opioid_deaths_con[opioid_deaths_con["policy_years"]>=0]

<ipython-input-17-82feb8123a57>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_buyers_con["policy_years"]= opioid_buyers_con["year"]-2010
<ipython-input-17-82feb8123a57>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_deaths_con["policy_years"]= opioid_deaths_con["Year"]-2010


In [18]:
# control buyers pre/post plot

before=alt.Chart(con_opi_before,title="Difference in Difference: Florida vs. Control, Opioids per cap").encode(x="policy_years", y="MME_per_cap")
base=before.transform_regression("policy_years","MME_per_cap").mark_line(color="red") 

fit, before_line = get_reg_fit(
    con_opi_before, yvar="MME_per_cap", xvar="policy_years", alpha=0.05, col="red")

fit, after_line = get_reg_fit(
    con_opi_after, yvar="MME_per_cap", xvar="policy_years", alpha=0.05, col="red")

con_chart_opioid=base+before_line+after_line+sep_line
#con_chart_opioid

In [19]:
diff_fl_opioid=con_chart_opioid+fl_chart_opioid
diff_fl_opioid

alt.LayerChart(...)

In [20]:
# control deaths pre/post plot
fit, before_line = get_reg_fit(
    con_death_before, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05,col="red"
)
fit, after_line = get_reg_fit(
    con_death_after, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05,col="red"
)

before=alt.Chart(con_death_before,title="Difference in Difference: Florida vs. Control, Overdose Deaths per cap").encode(x="policy_years", y="deaths_per_cap")
base=before.transform_regression("policy_years","deaths_per_cap").mark_line(color="#FFAA00") 

con_chart_death=base+before_line+after_line+sep_line

In [21]:
#Difference in difference plot, deaths, FL vs control
diff_fl_death=con_chart_death+fl_chart_death+sep_line
diff_fl_death

alt.LayerChart(...)

# Texas Plots

In [22]:
# creating texas dataframes

opioid_deaths_tx=opioid_deaths[opioid_deaths["STNAME"]=="Texas"]

opioid_deaths_tx["policy_years"]= opioid_deaths_tx["Year"]-2007


tx_death_before=opioid_deaths_tx[opioid_deaths_tx["policy_years"]<0]
tx_death_after=opioid_deaths_tx[opioid_deaths_tx["policy_years"]>=0]



<ipython-input-22-20e4429c7789>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_deaths_tx["policy_years"]= opioid_deaths_tx["Year"]-2007


In [23]:
# Creating texas control dataframes
states = ["Oklahoma","Lousiana","New Mexico"]

opioid_deaths_con=opioid_deaths[opioid_deaths["STNAME"].isin(states)]
opioid_deaths_con["policy_years"]= opioid_deaths_con["Year"]-2007


con_death_before=opioid_deaths_con[opioid_deaths_con["policy_years"]<0]
con_death_after=opioid_deaths_con[opioid_deaths_con["policy_years"]>=0]

<ipython-input-23-10a9ea96dd5f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_deaths_con["policy_years"]= opioid_deaths_con["Year"]-2007


In [24]:
# texas deaths pre/post plot
before=alt.Chart(tx_death_before,title="Pre-Post Policy: Texas, Overdose Deaths per cap").encode(x="policy_years", y="deaths_per_cap")
base=before.transform_regression("policy_years","deaths_per_cap").mark_line() 

fit, before_line = get_reg_fit(
    tx_death_before, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05
)
fit, after_line = get_reg_fit(
    tx_death_after, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05
)

tx_chart_death=base+before_line+after_line+sep_line
tx_chart_death

alt.LayerChart(...)

In [25]:
# control deaths pre/post plot
fit, before_line = get_reg_fit(
    con_death_before, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05,col="red"
)
fit, after_line = get_reg_fit(
    con_death_after, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05,col="red"
)

before=alt.Chart(con_death_before,title="Difference in Difference: Texas vs. Control, Overdose Deaths per cap").encode(x="policy_years", y="deaths_per_cap")
base=before.transform_regression("policy_years","deaths_per_cap").mark_line(color="#FFAA00") 

con_chart_death=base+before_line+after_line+sep_line

diff_tx_death=con_chart_death+tx_chart_death+sep_line
diff_tx_death

alt.LayerChart(...)

# Washington Plots

In [26]:
# creating Washington dataframes

opioid_deaths_wa=opioid_deaths[opioid_deaths["STNAME"]=="Washington"]

opioid_deaths_wa["policy_years"]= opioid_deaths_wa["Year"]-2012


wa_death_before=opioid_deaths_wa[opioid_deaths_wa["policy_years"]<0]
wa_death_after=opioid_deaths_wa[opioid_deaths_wa["policy_years"]>=0]



<ipython-input-26-e90579bb2b81>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_deaths_wa["policy_years"]= opioid_deaths_wa["Year"]-2012


In [27]:
# Creating wa control dataframes
states = ["Idaho","Oregon","Montana"]

opioid_deaths_con=opioid_deaths[opioid_deaths["STNAME"].isin(states)]
opioid_deaths_con["policy_years"]= opioid_deaths_con["Year"]-2012


con_death_before=opioid_deaths_con[opioid_deaths_con["policy_years"]<0]
con_death_after=opioid_deaths_con[opioid_deaths_con["policy_years"]>=0]

<ipython-input-27-06db1c6d3ea8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_deaths_con["policy_years"]= opioid_deaths_con["Year"]-2012


In [28]:
# washington deaths pre/post plot
before=alt.Chart(wa_death_before,title="Pre-Post Policy: Washington, Overdose Deaths per cap").encode(x="policy_years", y="deaths_per_cap")
base=before.transform_regression("policy_years","deaths_per_cap").mark_line() 

fit, before_line = get_reg_fit(
    wa_death_before, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05
)
fit, after_line = get_reg_fit(
    wa_death_after, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05
)

wa_chart_death=base+before_line+after_line+sep_line
wa_chart_death

alt.LayerChart(...)

In [29]:
# control deaths pre/post plot
fit, before_line = get_reg_fit(
    con_death_before, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05,col="red"
)
fit, after_line = get_reg_fit(
    con_death_after, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05,col="red"
)

before=alt.Chart(con_death_before,title="Difference in Difference: Washington vs. Control, Overdose Deaths per cap").encode(x="policy_years", y="deaths_per_cap")
base=before.transform_regression("policy_years","deaths_per_cap").mark_line(color="#FFAA00") 

con_chart_death=base+before_line+after_line+sep_line

diff_wa_death=con_chart_death+wa_chart_death+sep_line
diff_wa_death

alt.LayerChart(...)

In [30]:
## Creating Texas plots by month

opi_tx_data = pd.read_parquet("/Users/preetkhowaja/Documents/MIDS Fall 2021/Practicing DS/pds2021-opioids-pds6/10_modified_data/opi_tx_pop.parquet")

In [46]:
opi_tx_data['year'] = opi_tx_data['year'].astype(str)
opi_tx_data['month'] = opi_tx_data['month'].astype(str)

In [53]:
# Creating date variable to plot against
opi_tx_data.head()

opi_tx_data['date'] = opi_tx_data['year'] + '/' + opi_tx_data['month']
opi_tx_data['date'] = pd.to_datetime(opi_tx_data['date'])


,BUYER_STATE,BUYER_COUNTY,year,month,MME,Population,MME_per_cap,date
index,,,,,,,,
0,LA,ACADIA,2006,1,2.821391e+05,60522,4.661761,2006-01-01
1,LA,ACADIA,2006,2,1.607831e+06,60522,26.566053,2006-02-01
2,LA,ACADIA,2006,3,1.971931e+06,60522,32.582050,2006-03-01
3,LA,ACADIA,2006,4,1.578223e+06,60522,26.076841,2006-04-01
4,LA,ACADIA,2006,5,1.914167e+06,60522,31.627626,2006-05-01


In [69]:
# creating texas dataframes
opi_tx_data['year'] = pd.to_numeric(opi_tx_data['year'])
texas_plotting_data=opi_tx_data[opi_tx_data["BUYER_STATE"]=="TX"]

opi_tx_data["policy_months"]= opi_tx_data["year"]-2007
opi_tx_data.head()

# tx_death_before=opioid_deaths_tx[opioid_deaths_tx["policy_years"]<0]
# tx_death_after=opioid_deaths_tx[opioid_deaths_tx["policy_years"]>=0]


,BUYER_STATE,BUYER_COUNTY,year,month,MME,Population,MME_per_cap,date,policy_years
index,,,,,,,,,
0,LA,ACADIA,2006,1,2.821391e+05,60522,4.661761,2006-01-01,-1
1,LA,ACADIA,2006,2,1.607831e+06,60522,26.566053,2006-02-01,-1
2,LA,ACADIA,2006,3,1.971931e+06,60522,32.582050,2006-03-01,-1
3,LA,ACADIA,2006,4,1.578223e+06,60522,26.076841,2006-04-01,-1
4,LA,ACADIA,2006,5,1.914167e+06,60522,31.627626,2006-05-01,-1
